In [2]:
# 2.1
import gmpy2
from gmpy2 import mpz, isqrt, next_prime

def generate_large_prime(num_digits):
    lower_bound = mpz(10 ** (num_digits - 1))
    upper_bound = mpz(10 ** num_digits) - 1
    start = gmpy2.mpz_random(gmpy2.random_state(), upper_bound - lower_bound) + lower_bound
    return next_prime(start)

print(generate_large_prime(20))


43730994843186856103


In [12]:
import gmpy2
from gmpy2 import mpz
import random

def generate_keypair(p, q):
    n = mpz(p) * mpz(q)
    phi = (mpz(p)-1) * (mpz(q)-1)

    # Choose e
    e = mpz(random.randint(2, phi-1))
    while gmpy2.gcd(e, phi) != 1:
        e = mpz(random.randint(2, phi-1))

    # Compute d
    d = gmpy2.invert(e, phi)

    return ((e, n), (d, n))

def encrypt(pk, plaintext):
    e, n = pk
    return [gmpy2.powmod(ord(char), e, n) for char in plaintext]

def decrypt(pk, ciphertext):
    d, n = pk
    return ''.join([chr(gmpy2.powmod(char, d, n)) for char in ciphertext])

In [16]:
pub_key, pr_key = generate_keypair(  
49063569001866738362682902190200372389707668290251564320901376367154042562632826557056299728753153340290076805682108903248703104367393391468965658529112445519628466136515313064863114870921176753811127517080381282419813355430704884645954353738524910744628807945111506282337476995473236538938026544292608972368017364137,   
373227069315924075013882263403566564500481338300278147348916639461074580060805610912992412344333934375436319343117403751305623053248368645504248941205727014894637895640857803456746395453916455692080979243204500072380365373363069098960240857167005969258169502181917315482204881768588033457135683058061460603299)

print("Public Key:", pub_key)
print("Private Key:", pr_key)

message = "Tudor Sclifos"
encrypted_message = encrypt(pub_key, message)
print("Encrypted Message:", encrypted_message)

decrypted_message = decrypt(pr_key, encrypted_message)
print("Decrypted Message:", decrypted_message)

Public Key: (mpz(16945850600825362107753472306201900563311623481304640737272167673439833682585925593737837566450790142722455920439271820364181938895900813734449011151526043651651455186974189052642392121102386263340370976865496646998967681821276480605896515226412484092329103062129827556979935027572220236799506751961546313072836893778125028728554085779810924648148729365651119798188544986712208000177794111191591948328119596168027314193046558820337307818458765059480664555556849329502954465614157839036832109007355222380940833817882625624043017710898330134377504843675694642692305459975977767730059723335775387241997914263669170065670548351087), mpz(18311852068746340941488534539762753439592268811233636815476399726572107109759955653489147765340730496516701871640469322664693565025948364301127021674029466548974237178735428818121976309833743776781744606787063390935982295149539321275561151669596180829759969255586580113245402687702920556553632056877582698272827467506652033783577051294272695106538605

In [13]:
import gmpy2
from gmpy2 import mpz
import random

def generate_key(p, g):
    # ElGamal algorithm
    pr_key = random.randint(2, p - 2)
    pub_key = mpz(gmpy2.powmod(g, pr_key, p))

    return pub_key, pr_key

def encrypt(message, p, g, public_key):
    k = random.randint(2, p - 2)
    c1 = mpz(gmpy2.powmod(g, k, p))
    c2 = [(ord(m) * gmpy2.powmod(public_key, k, p)) for m in message]
    return (c1, c2)

def decrypt(ciphertext, p, pr_key):
    a = ciphertext[0]
    b = ciphertext[1]
    s = gmpy2.powmod(a, pr_key, p)
    s_1 = gmpy2.invert(s, p)
    message = ''.join([chr((char * s_1) % p) for char in b])
    return message

p = mpz(32317006071311007300153513477825163362488057133489075174588434139269806834136210002792056362640164685458556357935330816928829023080573472625273554742461245741026202527916572972862706300325263428213145766931414223654220941111348629991657478268034230553086349050635557712219187890332729569696129743856241741236237225197346402691855797767976823014625397933058015226858730761197532436467475855460715043896844940366130497697812854295958659597567051283852132784468522925504568272879113720098931873959143374175837826000278034973198552060607533234122603254684088120031105907484281003994966956119696956248629032338072839127039)
g = mpz(2)
pub_key, pr_key = generate_key(p, g)

message = "Tudor Sclifos"
encrypted_message = encrypt(message, p, g, pub_key)
print("Encrypted Message:", encrypted_message)

decrypted_message = decrypt(encrypted_message, p, pr_key)
print("Decrypted Message:", decrypted_message)


Encrypted Message: (mpz(8430685376686194900967753405372974793676960471920933842233344580474958713375971473961179357596667043953958839067891762332530862269617870310370547887185903228752191912255826235577685171605963178567625231776285498079695085094608465264685280966312903210580199269964529869636348728658863799046804613800513975070515431421434972139671222928566545985042625483120764813047025887506141022186484550936327377643583208016784414606152637826931279318942259987857661197519267687331102802782857929105368069756105763762843973733224688793488091342481528943873308994648086809912992577032240796844109474596860699498828947405872684696102), [mpz(6303118475982671473553273104440513868102524859619344556458889821178275039909072419351702877774188102276441754470887627387765427355640587852457078311215457167581869911326756888628184330431753119088088642975034400087817109139821250614207780168507072068846512583712344302361413955236374552137634552090738918238416136204354463111176518987251876397595675841

In [32]:
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives import padding
import os

def encrypt_aes(key, plaintext):
    # Generate a random IV (Initialization Vector)
    iv = os.urandom(16)
    
    # Pad the plaintext
    padder = padding.PKCS7(128).padder()
    padded_data = padder.update(plaintext.encode()) + padder.finalize()
    
    # Create an AES cipher object with CBC mode
    cipher = Cipher(algorithms.AES(key), modes.CFB(iv), backend=default_backend())
    encryptor = cipher.encryptor()
    
    # Encrypt the padded data
    ciphertext = encryptor.update(padded_data) + encryptor.finalize()
    
    # Return the IV and ciphertext
    return iv, ciphertext

def decrypt_aes(key, iv, ciphertext):
    # Create an AES cipher object with CBC mode
    cipher = Cipher(algorithms.AES(key), modes.CFB(iv), backend=default_backend())
    decryptor = cipher.decryptor()
    
    # Decrypt the ciphertext
    decrypted_data = decryptor.update(ciphertext) + decryptor.finalize()
    
    # Unpad the decrypted data
    unpadder = padding.PKCS7(128).unpadder()
    plaintext = unpadder.update(decrypted_data) + unpadder.finalize()
    
    # Return the decrypted plaintext
    return plaintext.decode()

def send_message(message):
    # Generate a random key for AES-256
    key = os.urandom(32)
    
    # Encrypt the message
    iv, ciphertext = encrypt_aes(key, message)
    
    # Decrypt the message
    decrypted_message = decrypt_aes(key, iv, ciphertext)

    return decrypted_message


Encrypted Message: 5def2896dbe87291e20e95a312af6514
Decrypted Message: 0000000


In [33]:
alice_secret_key = random.randint(2, 10000000)
bob_secret_key = random.randint(2, 10000000)

# Diffie-Hellman Key Exchange
g = 2
p = 32317006071311007300153513477825163362488057133489075174588434139269806834136210002792056362640164685458556357935330816928829023080573472625273554742461245741026202527916572972862706300325263428213145766931414223654220941111348629991657478268034230553086349050635557712219187890332729569696129743856241741236237225197346402691855797767976823014625397933058015226858730761197532436467475855460715043896844940366130497697812854295958659597567051283852132784468522925504568272879113720098931873959143374175837826000278034973198552060607533234122603254684088120031105907484281003994966956119696956248629032338072839127039

# Alice generates her public key
alice_public_key = pow(g, alice_secret_key, p)

# Bob generates his public key
bob_public_key = pow(g, bob_secret_key, p)

# Alice send her public key to Bob and Bob sends his public key to Alice
bobs_public_key_to_alice = send_message(str(bob_public_key))
alices_public_key_to_bob = send_message(str(alice_public_key))

# Alice and Bob exchange their public keys and compute the shared secret key
alice_shared_secret_key = pow(int(bobs_public_key_to_alice), alice_secret_key, p)
bob_shared_secret_key = pow(int(alices_public_key_to_bob), bob_secret_key, p)

print("Alice's Shared Secret Key:", alice_shared_secret_key)
print("Bob's Shared Secret Key:", bob_shared_secret_key)

Alice's Shared Secret Key: 6907237184197635045400862498597830055019040691428130196925307727886577317250597587883335807588003109507884767756807425486917460555181394627103484389672597250298891511434215215947740358778745001326110350278021911839969381826963604920868872050746044637358390305114436024424448079297549374700943208525717905948352386889104260040662812493013131494223367564948903578895917025278561125555066814288638712728401826106321012453641235878375299649667059928130581661891408742425055023984746605310339164959321209628556756036701963521656611579230000775260324002639599443831297336976914980210500126216125972177213850286992422362598
Bob's Shared Secret Key: 6907237184197635045400862498597830055019040691428130196925307727886577317250597587883335807588003109507884767756807425486917460555181394627103484389672597250298891511434215215947740358778745001326110350278021911839969381826963604920868872050746044637358390305114436024424448079297549374700943208525717905948352386889104260040662812